# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import os
import http.client
from datetime import datetime, timedelta
import json
import time

# Variables

In [47]:
footballdata_wIndexes_path = os.path.abspath(os.getcwd())+"\\Java\\football-data_wIndexes.csv"
idx_mappings_dir = os.path.abspath(os.getcwd())+"\\Data\\footballAPI_idx_mappings\\englishpremierleague.csv"

footballdata_wTeamStats_outputPath = os.path.abspath(os.getcwd())+"\\Data\\footballdata_wTeamStats_englishpremierleague.csv"

api_dailyLimit = 7000

# Load Football-Data File with Indexes

In [21]:
footballdata_wIndexes = pd.read_csv(footballdata_wIndexes_path, delimiter=";", index_col=False)
footballdata_wIndexes = footballdata_wIndexes.loc[:, ~footballdata_wIndexes.columns.str.contains('^Unnamed')]
footballdata_wIndexes

,Season,Date,HomeTeamid,HomeTeamName,AwayTeamId,AwayTeamName,HomeGoals,AwayGoals,FullTimeResult,AverageHomeWinOdd,AverageDrawWinOdd,AverageAwayWinOdd,AvgOver2_5,AvgUnder2_5,Div
0,2010,14/08/2010,66,Aston Villa,48,West Ham,3,0,H,2.00,3.30,4.00,2.01,1.75,39
1,2010,14/08/2010,67,Blackburn,45,Everton,1,0,H,2.88,3.25,2.50,2.08,1.70,39
2,2010,14/08/2010,68,Bolton,36,Fulham,0,0,D,2.20,3.30,3.40,2.09,1.69,39
3,2010,14/08/2010,49,Chelsea,60,West Brom,6,0,H,1.17,7.00,17.00,1.49,2.49,39
4,2010,14/08/2010,746,Sunderland,54,Birmingham,2,2,D,2.10,3.30,3.60,2.16,1.65,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4533,2021,11/05/2022,63,Leeds,49,Chelsea,0,3,A,4.98,4.17,1.67,1.64,2.28,39
4534,2021,11/05/2022,46,Leicester,71,Norwich,3,0,H,1.44,4.90,7.02,1.61,2.34,39
4535,2021,11/05/2022,38,Watford,45,Everton,0,0,D,4.26,3.68,1.87,1.91,1.92,39
4536,2021,11/05/2022,39,Wolves,50,Man City,1,5,A,12.83,5.88,1.26,1.66,2.24,39


In [22]:
footballdata_wIndexes.count()

Season               4538
Date                 4538
HomeTeamid           4538
HomeTeamName         4538
AwayTeamId           4538
AwayTeamName         4538
HomeGoals            4538
AwayGoals            4538
FullTimeResult       4538
AverageHomeWinOdd    4538
AverageDrawWinOdd    4538
AverageAwayWinOdd    4538
AvgOver2_5           4538
AvgUnder2_5          4538
Div                  4538
dtype: int64

## Store Index Mapping

In [23]:
idxs = footballdata_wIndexes[['HomeTeamName','HomeTeamid','Div']].drop_duplicates().set_index('HomeTeamName')\
    .sort_values('HomeTeamName')
idxs

,HomeTeamid,Div
HomeTeamName,,
Arsenal,42,39
Aston Villa,66,39
Birmingham,54,39
Blackburn,67,39
Blackpool,1356,39
Bolton,68,39
Bournemouth,35,39
Brentford,55,39
Brighton,51,39


In [24]:
idxs.to_csv(idx_mappings_dir)

# Call API to get match Stats

In [25]:
keys_columns = [(['form'], 'form'),
        
        (['fixtures','wins','home'], 'home_wins'),
        (['fixtures','draws','home'], 'home_draws'),
        (['fixtures','loses','home'], 'home_loses'),
        (['fixtures','wins','away'], 'away_wins'),
        (['fixtures','draws','away'], 'away_draws'),
        (['fixtures','loses','away'], 'away_loses'),
       
       (['goals','for','total','home'], 'home_goals_for'),
       (['goals','for','total','away'], 'away_goals_for'),
       (['goals','for','total','total'], 'total_goals_for'),
       (['goals','for','average','home'], 'average_home_goals_for'),
       (['goals','for','average','away'], 'average_away_goals_for'),
       (['goals','for','average','total'], 'average_total_goals_for'),
       (['goals','for','minute','0-15','total'], 'goals_for_minute_0_15'),
       (['goals','for','minute','0-15','percentage'], 'goals_for_minute_0_15_percentage'),
       (['goals','for','minute','16-30','total'], 'goals_for_minute_16_30'),
       (['goals','for','minute','16-30','percentage'], 'goals_for_minute_16_30_percentage'),
       (['goals','for','minute','31-45','total'], 'goals_for_minute_31_45'),
       (['goals','for','minute','31-45','percentage'], 'goals_for_minute_31_45_percentage'),
       (['goals','for','minute','46-60','total'], 'goals_for_minute_46_60'),
       (['goals','for','minute','46-60','percentage'], 'goals_for_minute_46_60_percentage'),
       (['goals','for','minute','61-75','total'], 'goals_for_minute_61_75'),
       (['goals','for','minute','61-75','percentage'], 'goals_for_minute_61_75_percentage'),
       (['goals','for','minute','76-90','total'], 'goals_for_minute_76_90'),
       (['goals','for','minute','76-90','percentage'], 'goals_for_minute_76_90_percentage'),
       (['goals','for','minute','91-105','total'], 'goals_for_minute_91_105'),
       (['goals','for','minute','91-105','percentage'], 'goals_for_minute_91_105_percentage'),
        
       (['goals','against','total','home'], 'home_goals_against'),
       (['goals','against','total','away'], 'away_goals_against'),
       (['goals','against','total','total'], 'total_goals_against'),
       (['goals','against','average','home'], 'average_home_goals_against'),
       (['goals','against','average','away'], 'average_away_goals_against'),
       (['goals','against','average','total'], 'average_total_goals_against'),
       (['goals','against','minute','0-15','total'], 'goals_against_minute_0_15'),
       (['goals','against','minute','0-15','percentage'], 'goals_against_minute_0_15_percentage'),
       (['goals','against','minute','16-30','total'], 'goals_against_minute_16_30'),
       (['goals','against','minute','16-30','percentage'], 'goals_against_minute_16_30_percentage'),
       (['goals','against','minute','31-45','total'], 'goals_against_minute_31_45'),
       (['goals','against','minute','31-45','percentage'], 'goals_against_minute_31_45_percentage'),
       (['goals','against','minute','46-60','total'], 'goals_against_minute_46_60'),
       (['goals','against','minute','46-60','percentage'], 'goals_against_minute_46_60_percentage'),
       (['goals','against','minute','61-75','total'], 'goals_against_minute_61_75'),
       (['goals','against','minute','61-75','percentage'], 'goals_against_minute_61_75_percentage'),
       (['goals','against','minute','76-90','total'], 'goals_against_minute_76_90'),
       (['goals','against','minute','76-90','percentage'], 'goals_against_minute_76_90_percentage'),
       (['goals','against','minute','91-105','total'], 'goals_against_minute_91_105'),
       (['goals','against','minute','91-105','percentage'], 'goals_against_minute_91_105_percentage'),
        
       (['biggest','streak','wins'], 'biggest_win_streak'),
       (['biggest','streak','draws'], 'biggest_draw_streak'),
       (['biggest','streak','loses'], 'biggest_lose_streak'),
        
       (['biggest','wins','home'], 'biggest_home_win'),
       (['biggest','wins','away'], 'biggest_away_win'),
       (['biggest','loses','home'], 'biggest_home_lose'),
       (['biggest','loses','away'], 'biggest_away_lose'),
       (['biggest','goals','for','home'], 'biggest_home_goals_for'),
       (['biggest','goals','for','away'], 'biggest_away_goals_for'),
       (['biggest','goals','against','home'], 'biggest_home_goals_against'),
       (['biggest','goals','against','away'], 'biggest_away_goals_against'),
        
       (['clean_sheet','home'], 'home_clean_sheets'),
       (['clean_sheet','away'], 'away_clean_sheets'),
       (['clean_sheet','total'], 'total_clean_sheets'),
        
       (['failed_to_score','home'], 'failed_to_score_home'),
       (['failed_to_score','away'], 'failed_to_score_away'),
       (['failed_to_score','total'], 'failed_to_score_total'),
        
       (['penalty','scored','total'], 'penalties_scored'),
       (['penalty','scored','percentage'], 'penalties_scored_percentage'),
       (['penalty','missed','total'], 'penalties_missed'),
       (['penalty','missed','percentage'], 'penalties_missed_percentage'),
       (['penalty','total'], 'penalties_for'),
        
        (['lineups'], 'lineups'),
        
       (['cards','yellow','0-15','total'], 'yellow_cards_minute_0_15'),
       (['cards','yellow','0-15','percentage'], 'yellow_cards_minute_0_15_percentage'),
       (['cards','yellow','16-30','total'], 'yellow_cards_minute_16_30'),
       (['cards','yellow','16-30','percentage'], 'yellow_cards_minute_16_30_percentage'),
       (['cards','yellow','31-45','total'], 'yellow_cards_minute_31_45'),
       (['cards','yellow','31-45','percentage'], 'yellow_cards_minute_31_45_percentage'),
       (['cards','yellow','46-60','total'], 'yellow_cards_minute_46_60'),
       (['cards','yellow','46-60','percentage'], 'yellow_cards_minute_46_60_percentage'),
       (['cards','yellow','61-75','total'], 'yellow_cards_minute_61_75'),
       (['cards','yellow','61-75','percentage'], 'yellow_cards_minute_61_75_percentage'),
       (['cards','yellow','76-90','total'], 'yellow_cards_minute_76_90'),
       (['cards','yellow','76-90','percentage'], 'yellow_cards_minute_76_90_percentage'),
       (['cards','yellow','91-105','total'], 'yellow_cards_minute_91_105'),
       (['cards','yellow','91-105','percentage'], 'yellow_cards_minute_91_105_percentage'),
        
       (['cards','red','0-15','total'], 'red_cards_minute_0_15'),
       (['cards','red','0-15','percentage'], 'red_cards_minute_0_15_percentage'),
       (['cards','red','16-30','total'], 'red_cards_minute_16_30'),
       (['cards','red','16-30','percentage'], 'red_cards_minute_16_30_percentage'),
       (['cards','red','31-45','total'], 'red_cards_minute_31_45'),
       (['cards','red','31-45','percentage'], 'red_cards_minute_31_45_percentage'),
       (['cards','red','46-60','total'], 'red_cards_minute_46_60'),
       (['cards','red','46-60','percentage'], 'red_cards_minute_46_60_percentage'),
       (['cards','red','61-75','total'], 'red_cards_minute_61_75'),
       (['cards','red','61-75','percentage'], 'red_cards_minute_61_75_percentage'),
       (['cards','red','76-90','total'], 'red_cards_minute_76_90'),
       (['cards','red','76-90','percentage'], 'red_cards_minute_76_90_percentage'),
       (['cards','red','91-105','total'], 'red_cards_minute_91_105'),
       (['cards','red','91-105','percentage'], 'red_cards_minute_91_105_percentage')]

In [44]:
pd.read_csv(footballdata_wTeamStats_outputPath, index_col=0)

,Unnamed: 0.1,AverageAwayWinOdd,AverageDrawWinOdd,AverageHomeWinOdd,AvgOver2_5,AvgUnder2_5,AwayGoals,AwayTeamId,AwayTeamName,Date,Div,FullTimeResult,HomeGoals,HomeTeamName,HomeTeamid,Season,away_average_away_goals_against,away_average_away_goals_for,away_average_home_goals_against,away_average_home_goals_for,away_average_total_goals_against,away_average_total_goals_for,away_away_clean_sheets,away_away_draws,away_away_goals_against,away_away_goals_for,away_away_loses,away_away_wins,away_biggest_away_goals_against,away_biggest_away_goals_for,away_biggest_away_lose,away_biggest_away_win,away_biggest_draw_streak,away_biggest_home_goals_against,away_biggest_home_goals_for,away_biggest_home_lose,away_biggest_home_win,away_biggest_lose_streak,away_biggest_win_streak,away_failed_to_score_away,away_failed_to_score_home,away_failed_to_score_total,away_form,away_goals_against_minute_0_15,away_goals_against_minute_0_15_percentage,away_goals_against_minute_16_30,away_goals_against_minute_16_30_percentage,away_goals_against_minute_31_45,away_goals_against_minute_31_45_percentage,away_goals_against_minute_46_60,away_goals_against_minute_46_60_percentage,away_goals_against_minute_61_75,away_goals_against_minute_61_75_percentage,away_goals_against_minute_76_90,away_goals_against_minute_76_90_percentage,away_goals_against_minute_91_105,away_goals_against_minute_91_105_percentage,away_goals_for_minute_0_15,away_goals_for_minute_0_15_percentage,away_goals_for_minute_16_30,away_goals_for_minute_16_30_percentage,away_goals_for_minute_31_45,away_goals_for_minute_31_45_percentage,away_goals_for_minute_46_60,away_goals_for_minute_46_60_percentage,away_goals_for_minute_61_75,away_goals_for_minute_61_75_percentage,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_goals_for_minute_91_105,away_goals_for_minute_91_105_percentage,away_home_clean_sheets,away_home_draws,away_home_goals_against,away_home_goals_for,away_home_loses,away_home_wins,away_lineups,away_penalties_for,away_penalties_missed,away_penalties_missed_percentage,away_penalties_scored,away_penalties_scored_percentage,away_red_cards_minute_0_15,away_red_cards_minute_0_15_percentage,away_red_cards_minute_16_30,away_red_cards_minute_16_30_percentage,away_red_cards_minute_31_45,away_red_cards_minute_31_45_percentage,away_red_cards_minute_46_60,away_red_cards_minute_46_60_percentage,away_red_cards_minute_61_75,away_red_cards_minute_61_75_percentage,away_red_cards_minute_76_90,away_red_cards_minute_76_90_percentage,away_red_cards_minute_91_105,away_red_cards_minute_91_105_percentage,away_total_clean_sheets,away_total_goals_against,away_total_goals_for,away_yellow_cards_minute_0_15,away_yellow_cards_minute_0_15_percentage,away_yellow_cards_minute_16_30,away_yellow_cards_minute_16_30_percentage,away_yellow_cards_minute_31_45,away_yellow_cards_minute_31_45_percentage,away_yellow_cards_minute_46_60,away_yellow_cards_minute_46_60_percentage,away_yellow_cards_minute_61_75,away_yellow_cards_minute_61_75_percentage,away_yellow_cards_minute_76_90,away_yellow_cards_minute_76_90_percentage,away_yellow_cards_minute_91_105,away_yellow_cards_minute_91_105_percentage,home_average_away_goals_against,home_average_away_goals_for,home_average_home_goals_against,home_average_home_goals_for,home_average_total_goals_against,home_average_total_goals_for,home_away_clean_sheets,home_away_draws,home_away_goals_against,home_away_goals_for,home_away_loses,home_away_wins,home_biggest_away_goals_against,home_biggest_away_goals_for,home_biggest_away_lose,home_biggest_away_win,home_biggest_draw_streak,home_biggest_home_goals_against,home_biggest_home_goals_for,home_biggest_home_lose,home_biggest_home_win,home_biggest_lose_streak,home_biggest_win_streak,home_failed_to_score_away,home_failed_to_score_home,home_failed_to_score_total,home_form,home_goals_against_minute_0_15,home_goals_against_minute_0_15_percentage,home_goals_against_minute_16_30,home_goals_against_minute_16_30_percentage,home_goals_against_minut

In [52]:
try:
    footballdata_wTeamStats = pd.read_csv(footballdata_wTeamStats_outputPath, index_col=0)
except FileNotFoundError:
    footballdata_wTeamStats = pd.DataFrame()

start_idx = len(footballdata_wTeamStats)
print(start_idx)

count = 0
for index, row in footballdata_wIndexes[start_idx:].iterrows():
    
    try:
        
        # subtract 1 day to get stats prior to the match
        date = (datetime.strptime(row['Date'], '%d/%m/%Y')-timedelta(days=1)).strftime("%Y-%m-%d")
        home_id = row['HomeTeamid']
        away_id = row['AwayTeamId']
        league_id = row['Div']
        season = row['Season']

        # api connection
        conn = http.client.HTTPSConnection("v3.football.api-sports.io")
        headers = {
            'x-rapidapi-host': "v3.football.api-sports.io",
            'x-rapidapi-key': "15f514ec0209e0f49397240d5e166bd9"
            }
        data = []

        # home team
        conn.request("GET",
             f"https://v3.football.api-sports.io/teams/statistics?league={league_id}&season={season}&team={home_id}&date={date}",
             headers=headers)
        res = conn.getresponse()
        data = res.read()
        home_data = json.loads(data.decode("utf-8"))
        count+=1

        # away team
        conn.request("GET",
             f"https://v3.football.api-sports.io/teams/statistics?league={league_id}&season={season}&team={away_id}&date={date}",
             headers=headers)
        res = conn.getresponse()
        data = res.read()
        away_data = json.loads(data.decode("utf-8"))
        count+=1

        values = []
        columns = []
        for i in range(2):
            if i == 0:
                team_stats = home_data
                header_ini = 'home_'
            elif i == 1:
                team_stats = away_data
                header_ini = 'away_'

            for (keys, column) in keys_columns:
                value = team_stats['response']
                for key in keys:
                    value = value[key]
                columns.append(header_ini+column)
                values.append(value)

        row = row.append(pd.Series(values, index=columns))

        footballdata_wTeamStats = footballdata_wTeamStats.append(row, ignore_index=True)

        # don't exceed api daily limit
        if count >= api_dailyLimit:
            break
        elif count%1000 == 0:
            print(f"We're here: Daily Requests:{int(count)}, Table Size:{index}")

        time.sleep(0.4)

    except Exception as e:
        print(home_data)
        print(away_data)
        print(e)
        print(f"We stopped here: {int(count)}, {index}")
        break
        
footballdata_wTeamStats.to_csv(footballdata_wTeamStats_outputPath)
footballdata_wTeamStats

23
We're here: Daily Requests:500, Table Size:522
We're here: Daily Requests:1000, Table Size:1022
We're here: Daily Requests:1500, Table Size:1522
We're here: Daily Requests:2000, Table Size:2022
We're here: Daily Requests:2500, Table Size:2522
We're here: Daily Requests:3000, Table Size:3022
We're here: Daily Requests:3500, Table Size:3522


,AverageAwayWinOdd,AverageDrawWinOdd,AverageHomeWinOdd,AvgOver2_5,AvgUnder2_5,AwayGoals,AwayTeamId,AwayTeamName,Date,Div,FullTimeResult,HomeGoals,HomeTeamName,HomeTeamid,Season,away_average_away_goals_against,away_average_away_goals_for,away_average_home_goals_against,away_average_home_goals_for,away_average_total_goals_against,away_average_total_goals_for,away_away_clean_sheets,away_away_draws,away_away_goals_against,away_away_goals_for,away_away_loses,away_away_wins,away_biggest_away_goals_against,away_biggest_away_goals_for,away_biggest_away_lose,away_biggest_away_win,away_biggest_draw_streak,away_biggest_home_goals_against,away_biggest_home_goals_for,away_biggest_home_lose,away_biggest_home_win,away_biggest_lose_streak,away_biggest_win_streak,away_failed_to_score_away,away_failed_to_score_home,away_failed_to_score_total,away_form,away_goals_against_minute_0_15,away_goals_against_minute_0_15_percentage,away_goals_against_minute_16_30,away_goals_against_minute_16_30_percentage,away_goals_against_minute_31_45,away_goals_against_minute_31_45_percentage,away_goals_against_minute_46_60,away_goals_against_minute_46_60_percentage,away_goals_against_minute_61_75,away_goals_against_minute_61_75_percentage,away_goals_against_minute_76_90,away_goals_against_minute_76_90_percentage,away_goals_against_minute_91_105,away_goals_against_minute_91_105_percentage,away_goals_for_minute_0_15,away_goals_for_minute_0_15_percentage,away_goals_for_minute_16_30,away_goals_for_minute_16_30_percentage,away_goals_for_minute_31_45,away_goals_for_minute_31_45_percentage,away_goals_for_minute_46_60,away_goals_for_minute_46_60_percentage,away_goals_for_minute_61_75,away_goals_for_minute_61_75_percentage,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_goals_for_minute_91_105,away_goals_for_minute_91_105_percentage,away_home_clean_sheets,away_home_draws,away_home_goals_against,away_home_goals_for,away_home_loses,away_home_wins,away_lineups,away_penalties_for,away_penalties_missed,away_penalties_missed_percentage,away_penalties_scored,away_penalties_scored_percentage,away_red_cards_minute_0_15,away_red_cards_minute_0_15_percentage,away_red_cards_minute_16_30,away_red_cards_minute_16_30_percentage,away_red_cards_minute_31_45,away_red_cards_minute_31_45_percentage,away_red_cards_minute_46_60,away_red_cards_minute_46_60_percentage,away_red_cards_minute_61_75,away_red_cards_minute_61_75_percentage,away_red_cards_minute_76_90,away_red_cards_minute_76_90_percentage,away_red_cards_minute_91_105,away_red_cards_minute_91_105_percentage,away_total_clean_sheets,away_total_goals_against,away_total_goals_for,away_yellow_cards_minute_0_15,away_yellow_cards_minute_0_15_percentage,away_yellow_cards_minute_16_30,away_yellow_cards_minute_16_30_percentage,away_yellow_cards_minute_31_45,away_yellow_cards_minute_31_45_percentage,away_yellow_cards_minute_46_60,away_yellow_cards_minute_46_60_percentage,away_yellow_cards_minute_61_75,away_yellow_cards_minute_61_75_percentage,away_yellow_cards_minute_76_90,away_yellow_cards_minute_76_90_percentage,away_yellow_cards_minute_91_105,away_yellow_cards_minute_91_105_percentage,home_average_away_goals_against,home_average_away_goals_for,home_average_home_goals_against,home_average_home_goals_for,home_average_total_goals_against,home_average_total_goals_for,home_away_clean_sheets,home_away_draws,home_away_goals_against,home_away_goals_for,home_away_loses,home_away_wins,home_biggest_away_goals_against,home_biggest_away_goals_for,home_biggest_away_lose,home_biggest_away_win,home_biggest_draw_streak,home_biggest_home_goals_against,home_biggest_home_goals_for,home_biggest_home_lose,home_biggest_home_win,home_biggest_lose_streak,home_biggest_win_streak,home_failed_to_score_away,home_failed_to_score_home,home_failed_to_score_total,home_form,home_goals_against_minute_0_15,home_goals_against_minute_0_15_percentage,home_goals_against_minute_16_30,home_goals_against_minute_16_30_percentage,home_goals_against_minute_31_45,home_